In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [3]:
!pip install "protobuf==3.20.3"

In [4]:
import os
import shutil

base_path = "/kaggle/input/food41"  # your dataset
images_path = f"{base_path}/images"
meta_path = f"{base_path}/meta/meta"

output_root = "/kaggle/working/food101_split"
train_dir = f"{output_root}/train"
test_dir = f"{output_root}/test"

# Create output folders
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Read split files
with open(f"{meta_path}/train.txt") as f:
    train_files = [line.strip() for line in f]

with open(f"{meta_path}/test.txt") as f:
    test_files = [line.strip() for line in f]

# Move images into train/test folder structure
for fpath in train_files:
    cls = fpath.split("/")[0]
    os.makedirs(f"{train_dir}/{cls}", exist_ok=True)
    shutil.copy(f"{images_path}/{fpath}.jpg", f"{train_dir}/{cls}/")

for fpath in test_files:
    cls = fpath.split("/")[0]
    os.makedirs(f"{test_dir}/{cls}", exist_ok=True)
    shutil.copy(f"{images_path}/{fpath}.jpg", f"{test_dir}/{cls}/")


In [9]:
import tensorflow as tf

# Load the training data
train_ds = tf.keras.utils.image_dataset_from_directory(
    "/kaggle/working/food101_split/train",
    image_size=(224, 224),
    batch_size=32,         
    label_mode='categorical' 
)

# Load the test data
test_ds = tf.keras.utils.image_dataset_from_directory(
    "/kaggle/working/food101_split/test",
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical' 
)

Found 75750 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [10]:
from tensorflow.keras import layers, models


model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),

    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1), 
    layers.RandomZoom(0.1),     
    layers.Rescaling(1./255),
    
    layers.Conv2D(32, (3, 3), padding='same'), 
    layers.BatchNormalization(),     
    layers.Activation('relu'),                 
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Dropout(0.5), 
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(101, activation='softmax')
])

In [11]:
# Train
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_ds, 
    validation_data=test_ds, 
    epochs=10  # Start with 10 to see how it goes
)

Epoch 1/10


E0000 00:00:1765566661.291890     724 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/sequential_2_1/dropout_2_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2368/2368 ━━━━━━━━━━━━━━━━━━━━ 225s 93ms/step - accuracy: 0.0554 - loss: 4.2983 - val_accuracy: 0.0531 - val_loss: 4.5682
Epoch 2/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 217s 91ms/step - accuracy: 0.1367 - loss: 3.6879 - val_accuracy: 0.1104 - val_loss: 3.9761
Epoch 3/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 217s 92ms/step - accuracy: 0.1950 - loss: 3.3650 - val_accuracy: 0.1029 - val_loss: 4.2592
Epoch 4/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 218s 92ms/step - accuracy: 0.2429 - loss: 3.1251 - val_accuracy: 0.1202 - val_loss: 4.1852
Epoch 5/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 219s 93ms/step - accuracy: 0.2796 - loss: 2.9304 - val_accuracy: 0.2181 - val_loss: 3.2959
Epoch 6/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 220s 93ms/step - accuracy: 0.3152 - loss: 2.7730 - val_accuracy: 0.2627 - val_loss: 3.0670
Epoch 7/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 218s 92ms/step - accuracy: 0.3405 - loss: 2.6578 - val_accuracy: 0.1799 - val_loss: 3.7613
Epoch 8/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 217s 92ms/step - accuracy: 0.3635 - lo

In [12]:
# Save the entire model (architecture + weights)
model.save("food101_custom_cnn_31acc.keras")

In [15]:
# Define the scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,  
    patience=2, 
    min_lr=1e-6,
    verbose=1 
)
# Train with the scheduler
history = model.fit(
    train_ds, 
    validation_data=test_ds, 
    epochs=25,          
    initial_epoch = 10,
    callbacks=[lr_scheduler] 
)

Epoch 11/25
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 219s 92ms/step - accuracy: 0.4136 - loss: 2.3289 - val_accuracy: 0.2823 - val_loss: 3.0944 - learning_rate: 0.0010
Epoch 12/25
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 217s 92ms/step - accuracy: 0.4275 - loss: 2.2638 - val_accuracy: 0.2929 - val_loss: 2.9971 - learning_rate: 0.0010
Epoch 13/25
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 217s 92ms/step - accuracy: 0.4392 - loss: 2.2134 - val_accuracy: 0.3939 - val_loss: 2.4294 - learning_rate: 0.0010
Epoch 14/25
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 217s 92ms/step - accuracy: 0.4510 - loss: 2.1576 - val_accuracy: 0.3503 - val_loss: 2.6588 - learning_rate: 0.0010
Epoch 15/25
2367/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.4614 - loss: 2.1168
Epoch 15: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 216s 91ms/step - accuracy: 0.4614 - loss: 2.1168 - val_accuracy: 0.3913 - val_loss: 2.4836 - learning_rate: 0.0010
Epoch 16/25
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 217s 92ms/s

In [16]:
# Save the final, 51% accuracy model
model.save("food101_custom_FINAL_51acc.keras")